## Imports

In [ ]:
#!pip install --upgrade scikit-learn

In [49]:
import time
from IPython.display import clear_output
import numpy    as np
import pandas   as pd
import seaborn  as sb
import matplotlib.pyplot as plt
import sklearn  as skl

from sklearn import pipeline      # Pipeline
from sklearn import preprocessing # OrdinalEncoder, LabelEncoder
from sklearn import impute
from sklearn import compose
from sklearn import model_selection # train_test_split
from sklearn import metrics         # accuracy_score, balanced_accuracy_score, plot_confusion_matrix
from sklearn import set_config

set_config(display='diagram') # Useful for display the pipeline

print("Pandas  ", pd.__version__)
print("Sklearn ", skl.__version__) # Try to use 0.24

Pandas   1.2.4
Sklearn  0.24.1


## Get the dataset
- **CLOUD = True**: Download dataset from Kaggle. Necesary for cloud enviroments like COLAB. **Specify your [kaggle credentials](https://www.kaggle.com/docs/api)**.
- **CLOUD = False**: Get the dataset from your local machine. **Specify the data path**.

In [50]:
CLOUD = False

if CLOUD:
    import os
    os.environ['KAGGLE_USERNAME'] = "your_kaggle_username"
    os.environ['KAGGLE_KEY']      = "your_kaggle_api_key"  # See https://www.kaggle.com/docs/api
    #!pip install --upgrade kaggle
    !kaggle competitions download -c titanic
    DATA_PATH = "./"

else:
    DATA_PATH = "../../titanic/"

## Load data

In [51]:
df      = pd.read_csv(DATA_PATH + "train.csv", index_col='PassengerId')
df_test = pd.read_csv(DATA_PATH + "test.csv",  index_col='PassengerId')

print("Train DataFrame:", df.shape)
print("Test DataFrame: ", df_test.shape)

Train DataFrame: (891, 11)
Test DataFrame:  (418, 10)


## Check missings

In [4]:
df.isnull().sum()

Survived      0
Pclass        0
Name          0
Sex           0
Age         177
SibSp         0
Parch         0
Ticket        0
Fare          0
Cabin       687
Embarked      2
dtype: int64

In [5]:
 df_test.isnull().sum()

Pclass        0
Name          0
Sex           0
Age          86
SibSp         0
Parch         0
Ticket        0
Fare          1
Cabin       327
Embarked      0
dtype: int64

# Exercise 1 (2pts):
Extract the title (Mr, Mrs, ... ) from the "Name" column.

Tips:
- split(',')[1] to get the 2nd part, and remove the surnamename
- split('.')[0] to get the 1str part, and remove the name

In [52]:
# CODE HERE get_Title_from_Name funtion
# Create this function using lambda (not def)

get_Title_from_Name = None

# YOUR CODE HERE
get_Title_from_Name = lambda name: name.split(',')[1].split('.')[0].strip()

df['Title']      = df['Name'].map(get_Title_from_Name)
df_test['Title'] = df_test['Name'].map(get_Title_from_Name)

In [53]:
assert df['Title'].values[0] == "Mr"
assert df['Title'].values[1] == "Mrs"
assert df['Title'].values[2] == "Miss"

assert df_test['Title'].values[0] == "Mr"
assert df_test['Title'].values[1] == "Mrs"
assert df_test['Title'].values[414] == "Dona"

# Exercise 2 (1pts):
Apply the title_dictionary to get a better information about the title. You have to overwrite the Title variable.

In [54]:
title_dictionary = {
    "Capt": "Officer",
    "Col": "Officer",
    "Major": "Officer",
    "Jonkheer": "Royalty",
    "Don": "Royalty",
    "Sir" : "Royalty",
    "Dr": "Officer",
    "Rev": "Officer",
    "the Countess":"Royalty",
    "Mme": "Mrs",
    "Mlle": "Miss",
    "Ms": "Mrs",
    "Mr" : "Mr",
    "Mrs" : "Mrs",
    "Miss" : "Miss",
    "Master" : "Master",
    "Lady" : "Royalty"
}
title_dictionary

{'Capt': 'Officer',
 'Col': 'Officer',
 'Major': 'Officer',
 'Jonkheer': 'Royalty',
 'Don': 'Royalty',
 'Sir': 'Royalty',
 'Dr': 'Officer',
 'Rev': 'Officer',
 'the Countess': 'Royalty',
 'Mme': 'Mrs',
 'Mlle': 'Miss',
 'Ms': 'Mrs',
 'Mr': 'Mr',
 'Mrs': 'Mrs',
 'Miss': 'Miss',
 'Master': 'Master',
 'Lady': 'Royalty'}

In [55]:
# Use map to apply the prevous dict

# df["Title"] =  
# df_test["Title"] =

# YOUR CODE HERE
df["Title"] = df.Title.map(title_dictionary) 
df_test["Title"] = df_test.Title.map(title_dictionary)

In [56]:
assert df['Title'].values[886] == "Officer"
assert df_test['Title'].values[417] == "Master"

# Exercise OPTINAL (0pts):
Try to extract some information from the feature **Ticket**. Search on Internet if that colum has some kind of information.

In [11]:
df_test['Ticket']

PassengerId
892                 330911
893                 363272
894                 240276
895                 315154
896                3101298
               ...        
1305             A.5. 3236
1306              PC 17758
1307    SOTON/O.Q. 3101262
1308                359309
1309                  2668
Name: Ticket, Length: 418, dtype: object

# Exercise OPTIONAL (0pts):
Try to extract some information from the feature **Cabin**. Search on Internet if that colum has some kind of information.

In [12]:
df_test['Cabin']

PassengerId
892      NaN
893      NaN
894      NaN
895      NaN
896      NaN
        ... 
1305     NaN
1306    C105
1307     NaN
1308     NaN
1309     NaN
Name: Cabin, Length: 418, dtype: object

# Preprocessing
For X data, notice that...
- We drop Survived because is the target variable
- We drop Name because we have extracted the Title: Mr, Mrs, ...
- We drop Ticket because it has no information -> see df.Ticket.nunique()
- We drop Cabin because it has a lot of missings (77% are missings)

Then, we identify **numerical** variables and **categorical** variables,

In [57]:
x = df.drop(columns=["Survived", 'Name', 'Ticket', 'Cabin']) # X DATA (WILL BE TRAIN+VALID DATA)
y = df["Survived"] # 0 = No, 1 = Yes

x_test = df_test.drop(columns=['Name', 'Ticket', 'Cabin']) # # X_TEST DATA (NEW DATA)

In [58]:
cat_vars  = ['Sex', 'Embarked', 'Title']         # x.select_dtypes(include=[object]).columns.values.tolist()
num_vars  = ['Pclass', 'SibSp', 'Parch', 'Fare', 'Age'] # x.select_dtypes(exclude=[object]).columns.values.tolist()

print("\nNumerical features:\n", num_vars)
print("\nCategorical features:\n", cat_vars)


Numerical features:
 ['Pclass', 'SibSp', 'Parch', 'Fare', 'Age']

Categorical features:
 ['Sex', 'Embarked', 'Title']


# Exercise 3 (2pts):
Create a **ColumnTransformer for Tree Models**. You need to create 2 pipelines (one for numerical and other for categories). Remember:
- Categorical pipeline: Some SimpleImputer -> Some Encoder
- Numerical pipeline: Some SimpleImputer -> NO Encoder

In [61]:
# YOUR CODE HERE
num_4_treeModels = pipeline.Pipeline(steps=[
  ('imputer', impute.SimpleImputer(strategy='mean', add_indicator=False))
])

cat_4_treeModels = pipeline.Pipeline(steps=[
  ('imputer', impute.SimpleImputer(strategy='constant', fill_value='missing')),
  ('ordinal', preprocessing.OrdinalEncoder(categories='auto', handle_unknown='use_encoded_value', unknown_value=99999999)) 
  # ('ordinal', preprocessing.OrdinalEncoder(categories='auto', handle_unknown='ignore'))
])

tree_prepro = compose.ColumnTransformer(transformers=[
    ('num', num_4_treeModels, num_vars),
    ('cat', cat_4_treeModels, cat_vars),
], remainder='drop') # Drop other vars not specified in num_vars or cat_vars

tree_prepro

ColumnTransformer(transformers=[('num',
                                 Pipeline(steps=[('imputer', SimpleImputer())]),
                                 ['Pclass', 'SibSp', 'Parch', 'Fare', 'Age']),
                                ('cat',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(fill_value='missing',
                                                                strategy='constant')),
                                                 ('ordinal',
                                                  OrdinalEncoder(handle_unknown='use_encoded_value',
                                                                 unknown_value=99999999))]),
                                 ['Sex', 'Embarked', 'Title'])])

In [62]:
assert type(tree_prepro)      is compose._column_transformer.ColumnTransformer
assert type(num_4_treeModels) is pipeline.Pipeline
assert type(cat_4_treeModels) is pipeline.Pipeline
assert len(num_4_treeModels) == 1
assert len(cat_4_treeModels) == 2

# Exercise 4 (1pts):
1. Complete the diccionary with some Tree Models.
2. Then we put each model in a Pipeline where:
   - first is the prepocessing with the column Transformer
   - Then is the Tree model
3. Display the fullpipeline of the LGBMClassifier

In [24]:
#!pip install xgboost
#!pip install lightgbm
#!pip install catboost

In [63]:
from sklearn.tree          import DecisionTreeClassifier
from sklearn.ensemble      import RandomForestClassifier
from sklearn.ensemble      import ExtraTreesClassifier
from sklearn.ensemble      import AdaBoostClassifier
from sklearn.ensemble      import GradientBoostingClassifier
from sklearn.experimental  import enable_hist_gradient_boosting # Necesary for HistGradientBoostingClassifier
from sklearn.ensemble      import HistGradientBoostingClassifier
from xgboost               import XGBClassifier
from lightgbm              import LGBMClassifier
from catboost              import CatBoostClassifier

In [64]:
# YOUR CODE HERE
tree_classifiers = {
  "Decision Tree": DecisionTreeClassifier(),
  "Extra Trees":ExtraTreesClassifier(),
  "Random Forest":RandomForestClassifier(),
  "AdaBoost":AdaBoostClassifier(),
  "Skl GBM":GradientBoostingClassifier(),
  "Skl HistGBM":GradientBoostingClassifier(),
  "XGBoost":XGBClassifier(),
  "LightGBM":LGBMClassifier(),
  "CatBoost":CatBoostClassifier()
}

tree_classifiers = {name: pipeline.make_pipeline(tree_prepro, model) for name, model in tree_classifiers.items()}

tree_classifiers["LightGBM"]

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer())]),
                                                  ['Pclass', 'SibSp', 'Parch',
                                                   'Fare', 'Age']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(fill_value='missing',
                                                                                 strategy='constant')),
                                                                  ('ordinal',
                                                                   OrdinalEncoder(handle_unknown='use_encoded_value',
                                                                                  unknown_value=99999999))]),
                                                  ['Sex', 'Embarked',
                                                   'Title'])])),
                ('lgbmclassifier', LGBMClassifier())])

In [67]:
for pipe in tree_classifiers.values():
    assert type(pipe) is pipeline.Pipeline

# Exercise 5 (3pts):
Define a simple split validation strategy with:
- 80% for train
- 20% for validation
- With stratification
- random_state=0

And train all the models in a for loop

In [66]:
results = pd.DataFrame({'Model': [], 'Accuracy': [], 'Bal Acc.': [], 'Time': []})

# YOUR CODE HERE
x_train, x_val, y_train, y_val = model_selection.train_test_split(
    x, y,
    test_size=0.2,
    stratify = y,   
    random_state=42,
)

for model_name, model in tree_classifiers.items():

    start_time = time.time()
    model.fit(x_train, y_train)
    total_time = time.time() - start_time
    
    pred = model.predict(x_val)
    
    results = results.append({"Model":    model_name,
                              "Accuracy": metrics.accuracy_score(y_val, pred)*100,
                              "Bal Acc.": metrics.balanced_accuracy_score(y_val, pred)*100,
                              "Time":     total_time},
                              ignore_index=True)


results_ord = results.sort_values(by=['Accuracy'], ascending=False, ignore_index=True)
results_ord.index += 1 
results_ord.style.bar(subset=['Accuracy', 'Bal Acc.'], vmin=0, vmax=100, color='#5fba7d')

C:\Users\User\anaconda3\envs\ML\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[20:34:42] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Learning rate set to 0.008911
0:	learn: 0.6868404	total: 1.97ms	remaining: 1.96s
1:	learn: 0.6805085	total: 4.03ms	remaining: 2.01s
2:	learn: 0.6751704	total: 5.94ms	remaining: 1.98s
3:	learn: 0.6703740	total: 6.99ms	remaining: 1.74s
4:	learn: 0.6656806	total: 8.04ms	remaining: 1.6s
5:	learn: 0.6606047	total: 9.49ms	remaining: 1.57s
6:	learn: 0.6556054	total: 11ms	remaining: 1.56s
7:	learn: 0.6509279	total: 12.5ms	remaining: 1.55s
8:	learn: 0.6457662	total: 14.4ms	remaining: 1.58s
9:	learn: 0.6405683	total: 16.2ms	remaining: 1.6s
10:	learn: 0.6354775	total: 18.1ms	remaining: 1.62s
11:	learn: 0.6303684	total: 20ms	remaining: 1.65s
12:	learn: 0.6251300	total: 21.9ms	remaining: 1.66s


175:	learn: 0.3864025	total: 339ms	remaining: 1.59s
176:	learn: 0.3859702	total: 342ms	remaining: 1.59s
177:	learn: 0.3856480	total: 345ms	remaining: 1.59s
178:	learn: 0.3852487	total: 347ms	remaining: 1.59s
179:	learn: 0.3849844	total: 349ms	remaining: 1.59s
180:	learn: 0.3847312	total: 351ms	remaining: 1.59s
181:	learn: 0.3841833	total: 353ms	remaining: 1.59s
182:	learn: 0.3841210	total: 355ms	remaining: 1.58s
183:	learn: 0.3836842	total: 357ms	remaining: 1.58s
184:	learn: 0.3833269	total: 360ms	remaining: 1.58s
185:	learn: 0.3829029	total: 362ms	remaining: 1.59s
186:	learn: 0.3823731	total: 365ms	remaining: 1.58s
187:	learn: 0.3819112	total: 367ms	remaining: 1.58s
188:	learn: 0.3815219	total: 369ms	remaining: 1.58s
189:	learn: 0.3810605	total: 372ms	remaining: 1.58s
190:	learn: 0.3805800	total: 374ms	remaining: 1.58s
191:	learn: 0.3802035	total: 377ms	remaining: 1.58s
192:	learn: 0.3799453	total: 379ms	remaining: 1.58s
193:	learn: 0.3798983	total: 380ms	remaining: 1.58s
194:	learn: 

339:	learn: 0.3446947	total: 860ms	remaining: 1.67s
340:	learn: 0.3445804	total: 863ms	remaining: 1.67s
341:	learn: 0.3442501	total: 867ms	remaining: 1.67s
342:	learn: 0.3440184	total: 870ms	remaining: 1.67s
343:	learn: 0.3439350	total: 874ms	remaining: 1.67s
344:	learn: 0.3437388	total: 877ms	remaining: 1.67s
345:	learn: 0.3435465	total: 881ms	remaining: 1.67s
346:	learn: 0.3434063	total: 885ms	remaining: 1.66s
347:	learn: 0.3433455	total: 888ms	remaining: 1.66s
348:	learn: 0.3431853	total: 892ms	remaining: 1.66s
349:	learn: 0.3429959	total: 895ms	remaining: 1.66s
350:	learn: 0.3427435	total: 899ms	remaining: 1.66s
351:	learn: 0.3426173	total: 903ms	remaining: 1.66s
352:	learn: 0.3425060	total: 907ms	remaining: 1.66s
353:	learn: 0.3424600	total: 911ms	remaining: 1.66s
354:	learn: 0.3424376	total: 913ms	remaining: 1.66s
355:	learn: 0.3421706	total: 918ms	remaining: 1.66s
356:	learn: 0.3420016	total: 922ms	remaining: 1.66s
357:	learn: 0.3417560	total: 926ms	remaining: 1.66s
358:	learn: 

545:	learn: 0.3155966	total: 1.55s	remaining: 1.29s
546:	learn: 0.3153293	total: 1.55s	remaining: 1.28s
547:	learn: 0.3151797	total: 1.55s	remaining: 1.28s
548:	learn: 0.3150014	total: 1.55s	remaining: 1.28s
549:	learn: 0.3147614	total: 1.56s	remaining: 1.27s
550:	learn: 0.3145012	total: 1.56s	remaining: 1.27s
551:	learn: 0.3143934	total: 1.56s	remaining: 1.27s
552:	learn: 0.3143014	total: 1.56s	remaining: 1.26s
553:	learn: 0.3142264	total: 1.56s	remaining: 1.26s
554:	learn: 0.3141300	total: 1.56s	remaining: 1.25s
555:	learn: 0.3139862	total: 1.57s	remaining: 1.25s
556:	learn: 0.3138178	total: 1.57s	remaining: 1.25s
557:	learn: 0.3137082	total: 1.57s	remaining: 1.24s
558:	learn: 0.3135828	total: 1.57s	remaining: 1.24s
559:	learn: 0.3134092	total: 1.57s	remaining: 1.24s
560:	learn: 0.3133103	total: 1.58s	remaining: 1.23s
561:	learn: 0.3132325	total: 1.58s	remaining: 1.23s
562:	learn: 0.3131219	total: 1.58s	remaining: 1.23s
563:	learn: 0.3130919	total: 1.58s	remaining: 1.22s
564:	learn: 

710:	learn: 0.2973730	total: 1.9s	remaining: 771ms
711:	learn: 0.2972269	total: 1.9s	remaining: 769ms
712:	learn: 0.2972190	total: 1.9s	remaining: 766ms
713:	learn: 0.2971380	total: 1.91s	remaining: 763ms
714:	learn: 0.2971070	total: 1.91s	remaining: 761ms
715:	learn: 0.2970189	total: 1.91s	remaining: 759ms
716:	learn: 0.2969529	total: 1.92s	remaining: 756ms
717:	learn: 0.2969468	total: 1.92s	remaining: 753ms
718:	learn: 0.2969062	total: 1.92s	remaining: 751ms
719:	learn: 0.2968995	total: 1.92s	remaining: 748ms
720:	learn: 0.2968473	total: 1.93s	remaining: 746ms
721:	learn: 0.2967631	total: 1.93s	remaining: 743ms
722:	learn: 0.2966337	total: 1.93s	remaining: 741ms
723:	learn: 0.2964920	total: 1.94s	remaining: 739ms
724:	learn: 0.2964717	total: 1.94s	remaining: 737ms
725:	learn: 0.2963447	total: 1.95s	remaining: 734ms
726:	learn: 0.2962593	total: 1.95s	remaining: 732ms
727:	learn: 0.2961781	total: 1.95s	remaining: 730ms
728:	learn: 0.2961176	total: 1.96s	remaining: 727ms
729:	learn: 0.2

896:	learn: 0.2774367	total: 2.59s	remaining: 297ms
897:	learn: 0.2772538	total: 2.59s	remaining: 294ms
898:	learn: 0.2770667	total: 2.6s	remaining: 292ms
899:	learn: 0.2770415	total: 2.6s	remaining: 289ms
900:	learn: 0.2767496	total: 2.6s	remaining: 286ms
901:	learn: 0.2767331	total: 2.61s	remaining: 283ms
902:	learn: 0.2766475	total: 2.61s	remaining: 281ms
903:	learn: 0.2765574	total: 2.62s	remaining: 278ms
904:	learn: 0.2764209	total: 2.62s	remaining: 275ms
905:	learn: 0.2762397	total: 2.63s	remaining: 272ms
906:	learn: 0.2761710	total: 2.63s	remaining: 270ms
907:	learn: 0.2761263	total: 2.63s	remaining: 267ms
908:	learn: 0.2759001	total: 2.64s	remaining: 264ms
909:	learn: 0.2757393	total: 2.64s	remaining: 261ms
910:	learn: 0.2756018	total: 2.65s	remaining: 258ms
911:	learn: 0.2755229	total: 2.65s	remaining: 256ms
912:	learn: 0.2754616	total: 2.65s	remaining: 253ms
913:	learn: 0.2753451	total: 2.66s	remaining: 250ms
914:	learn: 0.2752816	total: 2.66s	remaining: 247ms
915:	learn: 0.2

,Model,Accuracy,Bal Acc.,Time
1,CatBoost,84.357542,82.140975,3.351040
2,Random Forest,82.681564,81.587615,0.399936
3,Extra Trees,81.005587,79.953887,0.356259
4,Skl GBM,81.005587,78.333333,0.118243
5,Skl HistGBM,81.005587,78.333333,0.146297
6,Decision Tree,80.446927,78.959157,0.038340
7,LightGBM,80.446927,78.959157,0.075446
8,AdaBoost,78.770950,78.135705,0.160486
9,XGBoost,78.770950,77.865613,0.096735


In [74]:
results_ord.index += 1 #whats happening here
print(results_ord.index)
results_ord.index += 1
results_ord.index

RangeIndex(start=6, stop=15, step=1)


RangeIndex(start=7, stop=16, step=1)

In [75]:
assert results_ord["Accuracy"].min() > 75
assert results_ord["Bal Acc."].min() > 75
assert len(results_ord) == 9

# Exercise 6 (3pts):
Define a 10 Fold cross validation strategy with:
- With stratification
- shuffle=True
- random_state=0

And train all the models in a for loop.

Tip you can use **[cross_val_predict](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_predict.html)** for both training and predict with 

In [76]:
# YOUR CODE HERE
from sklearn.model_selection import cross_val_predict

skf = model_selection.StratifiedKFold( #This cross-validation object is a variation of KFold that 
    #returns stratified folds.The folds are made by preserving the percentage of samples for each class.
    n_splits= 10,
    shuffle=True,
    random_state = 42
)

results = pd.DataFrame({'Model': [], 'Accuracy': [], 'Bal Acc.': [], 'Time': []})


for model_name, model in tree_classifiers.items():

    start_time = time.time()
    model.fit(x_train, y_train)
    total_time = time.time() - start_time
    
    pred = cross_val_predict(model, x_val, y_val, cv=skf)
    pred = model.predict(x_val) 
    
    results = results.append({"Model":    model_name,
                              "Accuracy": metrics.accuracy_score(y_val, pred)*100,
                              "Bal Acc.": metrics.balanced_accuracy_score(y_val, pred)*100,
                              "Time":     total_time},
                              ignore_index=True)


results_ord = results.sort_values(by=['Accuracy'], ascending=False, ignore_index=True)
results_ord.index += 1 
results_ord.style.bar(subset=['Accuracy', 'Bal Acc.'], vmin=0, vmax=100, color='#5fba7d')

C:\Users\User\anaconda3\envs\ML\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[20:45:45] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\Users\User\anaconda3\envs\ML\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
C:\Users\User\anaconda3\envs\ML\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[20:45:45] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[20:45:45] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\Users\User\anaconda3\envs\ML\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
C:\Users\User\anaconda3\envs\ML\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


...

316:	learn: 0.3821516	total: 1.16s	remaining: 2.49s
317:	learn: 0.3818351	total: 1.16s	remaining: 2.49s
318:	learn: 0.3813618	total: 1.16s	remaining: 2.48s
319:	learn: 0.3809257	total: 1.16s	remaining: 2.47s
320:	learn: 0.3806106	total: 1.17s	remaining: 2.47s
321:	learn: 0.3804986	total: 1.17s	remaining: 2.46s
322:	learn: 0.3804633	total: 1.17s	remaining: 2.45s
323:	learn: 0.3800758	total: 1.17s	remaining: 2.44s
324:	learn: 0.3796629	total: 1.17s	remaining: 2.44s
325:	learn: 0.3790915	total: 1.18s	remaining: 2.43s
326:	learn: 0.3787074	total: 1.18s	remaining: 2.43s
327:	learn: 0.3780583	total: 1.18s	remaining: 2.42s
328:	learn: 0.3774894	total: 1.18s	remaining: 2.41s
329:	learn: 0.3773427	total: 1.19s	remaining: 2.41s
330:	learn: 0.3768265	total: 1.19s	remaining: 2.4s
331:	learn: 0.3761949	total: 1.19s	remaining: 2.4s
332:	learn: 0.3757782	total: 1.19s	remaining: 2.39s
333:	learn: 0.3753689	total: 1.2s	remaining: 2.38s
334:	learn: 0.3750883	total: 1.2s	remaining: 2.38s
335:	learn: 0.37

552:	learn: 0.3210250	total: 1.66s	remaining: 1.34s
553:	learn: 0.3210043	total: 1.66s	remaining: 1.33s
554:	learn: 0.3207146	total: 1.66s	remaining: 1.33s
555:	learn: 0.3204322	total: 1.66s	remaining: 1.33s
556:	learn: 0.3199855	total: 1.67s	remaining: 1.32s
557:	learn: 0.3198372	total: 1.67s	remaining: 1.32s
558:	learn: 0.3196349	total: 1.67s	remaining: 1.32s
559:	learn: 0.3195558	total: 1.67s	remaining: 1.31s
560:	learn: 0.3193431	total: 1.68s	remaining: 1.31s
561:	learn: 0.3191876	total: 1.68s	remaining: 1.31s
562:	learn: 0.3190890	total: 1.68s	remaining: 1.3s
563:	learn: 0.3188787	total: 1.68s	remaining: 1.3s
564:	learn: 0.3188281	total: 1.69s	remaining: 1.3s
565:	learn: 0.3186640	total: 1.69s	remaining: 1.29s
566:	learn: 0.3184090	total: 1.69s	remaining: 1.29s
567:	learn: 0.3181735	total: 1.69s	remaining: 1.29s
568:	learn: 0.3179298	total: 1.7s	remaining: 1.28s
569:	learn: 0.3178697	total: 1.7s	remaining: 1.28s
570:	learn: 0.3177552	total: 1.7s	remaining: 1.28s
571:	learn: 0.3174

757:	learn: 0.2842047	total: 2.14s	remaining: 683ms
758:	learn: 0.2840593	total: 2.14s	remaining: 680ms
759:	learn: 0.2838059	total: 2.14s	remaining: 677ms
760:	learn: 0.2835971	total: 2.15s	remaining: 674ms
761:	learn: 0.2834693	total: 2.15s	remaining: 671ms
762:	learn: 0.2833301	total: 2.15s	remaining: 668ms
763:	learn: 0.2831300	total: 2.15s	remaining: 665ms
764:	learn: 0.2829814	total: 2.16s	remaining: 662ms
765:	learn: 0.2826815	total: 2.16s	remaining: 659ms
766:	learn: 0.2825834	total: 2.16s	remaining: 656ms
767:	learn: 0.2824182	total: 2.16s	remaining: 653ms
768:	learn: 0.2822454	total: 2.16s	remaining: 650ms
769:	learn: 0.2820087	total: 2.17s	remaining: 647ms
770:	learn: 0.2817903	total: 2.17s	remaining: 644ms
771:	learn: 0.2816315	total: 2.17s	remaining: 641ms
772:	learn: 0.2815805	total: 2.17s	remaining: 638ms
773:	learn: 0.2813537	total: 2.17s	remaining: 635ms
774:	learn: 0.2812199	total: 2.18s	remaining: 632ms
775:	learn: 0.2810888	total: 2.18s	remaining: 629ms
776:	learn: 

964:	learn: 0.2526414	total: 2.64s	remaining: 95.9ms
965:	learn: 0.2525487	total: 2.65s	remaining: 93.1ms
966:	learn: 0.2524296	total: 2.65s	remaining: 90.4ms
967:	learn: 0.2521756	total: 2.65s	remaining: 87.6ms
968:	learn: 0.2520350	total: 2.65s	remaining: 84.9ms
969:	learn: 0.2519880	total: 2.65s	remaining: 82.1ms
970:	learn: 0.2518640	total: 2.66s	remaining: 79.4ms
971:	learn: 0.2517783	total: 2.66s	remaining: 76.6ms
972:	learn: 0.2516671	total: 2.66s	remaining: 73.9ms
973:	learn: 0.2515328	total: 2.67s	remaining: 71.2ms
974:	learn: 0.2514860	total: 2.67s	remaining: 68.4ms
975:	learn: 0.2514489	total: 2.67s	remaining: 65.7ms
976:	learn: 0.2513437	total: 2.67s	remaining: 62.9ms
977:	learn: 0.2511988	total: 2.67s	remaining: 60.2ms
978:	learn: 0.2511485	total: 2.68s	remaining: 57.4ms
979:	learn: 0.2510776	total: 2.68s	remaining: 54.7ms
980:	learn: 0.2508697	total: 2.68s	remaining: 52ms
981:	learn: 0.2508091	total: 2.69s	remaining: 49.2ms
982:	learn: 0.2507218	total: 2.69s	remaining: 46

,Model,Accuracy,Bal Acc.,Time
1,CatBoost,84.357542,82.140975,3.850160
2,Random Forest,81.564246,80.138340,0.318176
3,Skl GBM,81.005587,78.333333,0.111161
4,Skl HistGBM,81.005587,78.333333,0.108093
5,Decision Tree,80.446927,78.959157,0.041422
6,LightGBM,80.446927,78.959157,0.066511
7,Extra Trees,79.888268,79.044796,0.293534
8,AdaBoost,78.770950,78.135705,0.191099
9,XGBoost,78.770950,77.865613,0.217129


In [77]:
assert results_ord["Accuracy"].min() > 75
assert results_ord["Bal Acc."].min() > 75
assert len(results_ord) == 9

# Exercise 7.1
Train with all data the best model

In [78]:
# best_model = tree_classifiers["SELECT MY BEST MODEL HERE"]

# Fit best model with all data


# YOUR CODE HERE
best_model = tree_classifiers["Skl HistGBM"]
best_model.fit(x,y)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer())]),
                                                  ['Pclass', 'SibSp', 'Parch',
                                                   'Fare', 'Age']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(fill_value='missing',
                                                                                 strategy='constant')),
                                                                  ('ordinal',
                                                                   OrdinalEncoder(handle_unknown='use_encoded_value',
                                                                                  unknown_value=99999999))]),
                                                  ['Sex', 'Embarked',
                                                   'Title'])])),
                ('gradientboostingclassifier', GradientBoostingClassifier())])

# Exercise 7.2 (2pts)
With your best model, generate the predicitions for test data (x_test)

In [79]:
# test_pred = # Get the predictions for x_test

# YOUR CODE HERE
test_pred = best_model.predict(x_test)

In [80]:
assert len(test_pred) == 418
assert np.unique(test_pred).tolist() == [0,1]

# Exercise 7.3

Submit to kaggle.

- You can use the kaggle command line app. Check https://github.com/Kaggle/kaggle-api

In [81]:
sub = pd.DataFrame(test_pred, index=x_test.index, columns=["Survived"])
sub.head()

,Survived
PassengerId,
892,0
893,0
894,0
895,0
896,1


In [82]:
sub.to_csv("sub.csv")

In [ ]:
!kaggle competitions submit -c titanic -f sub.csv -m "My submission message"